In [1]:
import numpy as np
import pandas as pd
import re
from tqdm import tqdm
import datetime as datetime
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("BPI_Challenge_2012.csv", parse_dates = ['time:timestamp'])
df.info()
# The default name indicating the case ID is case:concept:name
# concept:name is the event
# time:timestamp is the corresponding timestamp

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262200 entries, 0 to 262199
Data columns (total 8 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Unnamed: 0            262200 non-null  int64  
 1   org:resource          244190 non-null  float64
 2   lifecycle:transition  262200 non-null  object 
 3   concept:name          262200 non-null  object 
 4   time:timestamp        262200 non-null  object 
 5   case:REG_DATE         262200 non-null  object 
 6   case:concept:name     262200 non-null  int64  
 7   case:AMOUNT_REQ       262200 non-null  int64  
dtypes: float64(1), int64(3), object(4)
memory usage: 16.0+ MB


# 1. Splitting Data

In [3]:
# Obtain date (datetime format) from datatype of time:timestamp 
df['Date'] = np.array(df['time:timestamp'].values, dtype = 'datetime64[D]').astype(datetime.datetime)
df

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  


,Unnamed: 0,org:resource,lifecycle:transition,concept:name,time:timestamp,case:REG_DATE,case:concept:name,case:AMOUNT_REQ,Date
0,0,112.0,COMPLETE,A_SUBMITTED,2011-10-01 00:38:44.546000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,2011-09-30
1,1,112.0,COMPLETE,A_PARTLYSUBMITTED,2011-10-01 00:38:44.880000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,2011-09-30
2,2,112.0,COMPLETE,A_PREACCEPTED,2011-10-01 00:39:37.906000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,2011-09-30
3,3,112.0,SCHEDULE,W_Completeren aanvraag,2011-10-01 00:39:38.875000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,2011-09-30
4,4,NaN,START,W_Completeren aanvraag,2011-10-01 11:36:46.437000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,2011-10-01
...,...,...,...,...,...,...,...,...,...
262195,262195,112.0,COMPLETE,A_PARTLYSUBMITTED,2012-02-29 23:51:17.423000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000,2012-02-29
262196,262196,112.0,SCHEDULE,W_Afhandelen leads,2012-02-29 23:52:01.287000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000,2012-02-29
262197,262197,11169.0,START,W_Afhandelen leads,2012-03-01 09:26:46.736000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000,2012-03-01
262198,262198,11169.0,COMPLETE,A_DECLINED,2012-03-01 09:27:37.118000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000,2012-03-01


In [4]:
# Determine training and testing data's date boundaries
date_unique = sorted(df['Date'].unique())
total_date = len(date_unique)
all_train_nr = round(total_date * 0.8)
date_before_test = date_unique[all_train_nr - 1]
date_before_test

datetime.date(2012, 2, 10)

In [5]:
# Remove entries with case ID across date boundaries
small_df = df[['Date', 'case:concept:name']].drop_duplicates()
small_df_1 = small_df[small_df['Date'] <= date_before_test]
small_df_2 = small_df[small_df['Date'] > date_before_test]
bruh = set(small_df_1['case:concept:name'].unique()).intersection(set(small_df_2['case:concept:name'].unique()))
case_unique = sorted(list(set(small_df_1['case:concept:name'].unique()) - bruh))

In [6]:
# Determine training and testing data's ID bpundaries afer determining suitable IDs
total_case = len(case_unique)
all_train_case = round(total_case * 0.8)
case_all_train = case_unique[: all_train_case]
case_test = case_unique[all_train_case: ]

# Split training and validation dataset
case_train, case_val = train_test_split(case_all_train, test_size = 0.2)

# Split the dataset
df_train = df[df['case:concept:name'].isin(case_train)]
df_val = df[df['case:concept:name'].isin(case_val)]
df_test = df[df['case:concept:name'].isin(case_test)]
df_train = df_train.drop(columns = ['Unnamed: 0', 'Date']).reset_index(drop = True)
df_val = df_val.drop(columns = ['Unnamed: 0', 'Date']).reset_index(drop = True)
df_test = df_test.drop(columns = ['Unnamed: 0', 'Date']).reset_index(drop = True)

In [7]:
df_train

,org:resource,lifecycle:transition,concept:name,time:timestamp,case:REG_DATE,case:concept:name,case:AMOUNT_REQ
0,112.0,COMPLETE,A_SUBMITTED,2011-10-01 00:38:44.546000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000
1,112.0,COMPLETE,A_PARTLYSUBMITTED,2011-10-01 00:38:44.880000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000
2,112.0,COMPLETE,A_PREACCEPTED,2011-10-01 00:39:37.906000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000
3,112.0,SCHEDULE,W_Completeren aanvraag,2011-10-01 00:39:38.875000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000
4,NaN,START,W_Completeren aanvraag,2011-10-01 11:36:46.437000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000
...,...,...,...,...,...,...,...
130686,11202.0,COMPLETE,A_DECLINED,2012-01-10 16:49:30.661000+01:00,2012-01-09 17:37:28.337000+01:00,199339,25000
130687,NaN,COMPLETE,W_Completeren aanvraag,2012-01-10 16:49:34.144000+01:00,2012-01-09 17:37:28.337000+01:00,199339,25000
130688,112.0,COMPLETE,A_SUBMITTED,2012-01-09 17:37:34.249000+01:00,2012-01-09 17:37:34.249000+01:00,199342,11500
130689,112.0,COMPLETE,A_PARTLYSUBMITTED,2012-01-09 17:37:34.749000+01:00,2012-01-09 17:37:34.249000+01:00,199342,11500


In [8]:
df_val

,org:resource,lifecycle:transition,concept:name,time:timestamp,case:REG_DATE,case:concept:name,case:AMOUNT_REQ
0,112.0,COMPLETE,A_SUBMITTED,2011-10-01 09:45:37.274000+02:00,2011-10-01 09:45:37.274000+02:00,173706,18000
1,112.0,COMPLETE,A_PARTLYSUBMITTED,2011-10-01 09:45:37.363000+02:00,2011-10-01 09:45:37.274000+02:00,173706,18000
2,112.0,SCHEDULE,W_Afhandelen leads,2011-10-01 09:46:18.623000+02:00,2011-10-01 09:45:37.274000+02:00,173706,18000
3,10912.0,START,W_Afhandelen leads,2011-10-01 10:15:43.883000+02:00,2011-10-01 09:45:37.274000+02:00,173706,18000
4,10912.0,COMPLETE,A_PREACCEPTED,2011-10-01 10:16:48.956000+02:00,2011-10-01 09:45:37.274000+02:00,173706,18000
...,...,...,...,...,...,...,...
32245,10138.0,COMPLETE,W_Valideren aanvraag,2012-02-10 13:02:04.765000+01:00,2012-01-09 17:40:12.748000+01:00,199345,8500
32246,10138.0,START,W_Valideren aanvraag,2012-02-10 13:32:48.693000+01:00,2012-01-09 17:40:12.748000+01:00,199345,8500
32247,10138.0,COMPLETE,A_DECLINED,2012-02-10 13:41:38.050000+01:00,2012-01-09 17:40:12.748000+01:00,199345,8500
32248,10138.0,COMPLETE,O_DECLINED,2012-02-10 13:41:38.050000+01:00,2012-01-09 17:40:12.748000+01:00,199345,8500


In [9]:
df_test

,org:resource,lifecycle:transition,concept:name,time:timestamp,case:REG_DATE,case:concept:name,case:AMOUNT_REQ
0,112.0,COMPLETE,A_SUBMITTED,2012-01-09 17:44:18.199000+01:00,2012-01-09 17:44:18.199000+01:00,199348,6000
1,112.0,COMPLETE,A_PARTLYSUBMITTED,2012-01-09 17:44:18.401000+01:00,2012-01-09 17:44:18.199000+01:00,199348,6000
2,112.0,COMPLETE,A_PREACCEPTED,2012-01-09 17:44:55.414000+01:00,2012-01-09 17:44:18.199000+01:00,199348,6000
3,112.0,SCHEDULE,W_Completeren aanvraag,2012-01-09 17:44:55.822000+01:00,2012-01-09 17:44:18.199000+01:00,199348,6000
4,10929.0,START,W_Completeren aanvraag,2012-01-09 17:45:33+01:00,2012-01-09 17:44:18.199000+01:00,199348,6000
...,...,...,...,...,...,...,...
27257,112.0,COMPLETE,A_PARTLYSUBMITTED,2012-02-10 22:15:22.972000+01:00,2012-02-10 22:15:19.240000+01:00,208295,48000
27258,112.0,COMPLETE,A_DECLINED,2012-02-10 22:16:18.231000+01:00,2012-02-10 22:15:19.240000+01:00,208295,48000
27259,112.0,COMPLETE,A_SUBMITTED,2012-02-10 22:57:49.541000+01:00,2012-02-10 22:57:49.541000+01:00,208301,2500
27260,112.0,COMPLETE,A_PARTLYSUBMITTED,2012-02-10 22:57:49.866000+01:00,2012-02-10 22:57:49.541000+01:00,208301,2500


# 2. Baseline Time Prediction (Only on Training Dataset)

In [10]:
# Calculate time difference
df_train['time_diff'] = df_train['time:timestamp'].diff().dt.total_seconds()
# Set the time difference of the 1st row 0
df_train.loc[0, 'time_diff'] = 0
# Count number of processes per trace/ID
count_lst = df_train.groupby('case:concept:name').count()['time_diff'].tolist()
# Assign position number to each row/process
position_lst_1 = [list(range(1, i + 1)) for i in count_lst]
position_lst = []
for i in position_lst_1:
    for j in i:
        position_lst.append(j)
df_train['position'] = position_lst
# Set the time difference of every process with position = 1 as 0
df_train.loc[df_train['position'] == 1, 'time_diff'] = 0
df_train

,org:resource,lifecycle:transition,concept:name,time:timestamp,case:REG_DATE,case:concept:name,case:AMOUNT_REQ,time_diff,position
0,112.0,COMPLETE,A_SUBMITTED,2011-10-01 00:38:44.546000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,0.000,1
1,112.0,COMPLETE,A_PARTLYSUBMITTED,2011-10-01 00:38:44.880000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,0.334,2
2,112.0,COMPLETE,A_PREACCEPTED,2011-10-01 00:39:37.906000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,53.026,3
3,112.0,SCHEDULE,W_Completeren aanvraag,2011-10-01 00:39:38.875000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,0.969,4
4,NaN,START,W_Completeren aanvraag,2011-10-01 11:36:46.437000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,39427.562,5
...,...,...,...,...,...,...,...,...,...
130686,11202.0,COMPLETE,A_DECLINED,2012-01-10 16:49:30.661000+01:00,2012-01-09 17:37:28.337000+01:00,199339,25000,607.826,9
130687,NaN,COMPLETE,W_Completeren aanvraag,2012-01-10 16:49:34.144000+01:00,2012-01-09 17:37:28.337000+01:00,199339,25000,3.483,10
130688,112.0,COMPLETE,A_SUBMITTED,2012-01-09 17:37:34.249000+01:00,2012-01-09 17:37:34.249000+01:00,199342,11500,0.000,1
130689,112.0,COMPLETE,A_PARTLYSUBMITTED,2012-01-09 17:37:34.749000+01:00,2012-01-09 17:37:34.249000+01:00,199342,11500,0.500,2


In [11]:
# Calculate mean time difference grouped by position based on the number of cases
mean_time_lst = df_train.groupby('position').mean()['time_diff'].tolist()
# Delete 1st time difference mean (Position = 1 always has mean = 0)
del mean_time_lst[0]
# Add the last time differnce mean as 0 when it reaches the maximum value of position
mean_time_lst.append(0.0)
# Create the predicted time column per entry using the mean time difference
pred_time_lst = [mean_time_lst[j - 1] for j in position_lst]
df_train['baseline_predicted_time'] = pred_time_lst
df_train

,org:resource,lifecycle:transition,concept:name,time:timestamp,case:REG_DATE,case:concept:name,case:AMOUNT_REQ,time_diff,position,baseline_predicted_time
0,112.0,COMPLETE,A_SUBMITTED,2011-10-01 00:38:44.546000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,0.000,1,0.557199
1,112.0,COMPLETE,A_PARTLYSUBMITTED,2011-10-01 00:38:44.880000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,0.334,2,35.557763
2,112.0,COMPLETE,A_PREACCEPTED,2011-10-01 00:39:37.906000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,53.026,3,9691.845836
3,112.0,SCHEDULE,W_Completeren aanvraag,2011-10-01 00:39:38.875000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,0.969,4,11553.950361
4,NaN,START,W_Completeren aanvraag,2011-10-01 11:36:46.437000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,39427.562,5,1227.997394
...,...,...,...,...,...,...,...,...,...,...
130686,11202.0,COMPLETE,A_DECLINED,2012-01-10 16:49:30.661000+01:00,2012-01-09 17:37:28.337000+01:00,199339,25000,607.826,9,11062.778748
130687,NaN,COMPLETE,W_Completeren aanvraag,2012-01-10 16:49:34.144000+01:00,2012-01-09 17:37:28.337000+01:00,199339,25000,3.483,10,12777.793761
130688,112.0,COMPLETE,A_SUBMITTED,2012-01-09 17:37:34.249000+01:00,2012-01-09 17:37:34.249000+01:00,199342,11500,0.000,1,0.557199
130689,112.0,COMPLETE,A_PARTLYSUBMITTED,2012-01-09 17:37:34.749000+01:00,2012-01-09 17:37:34.249000+01:00,199342,11500,0.500,2,35.557763


# 3. Apply Above Calculated Mean Time to Validation and Test Set

In [12]:
# Calculate time difference
df_val['time_diff'] = df_val['time:timestamp'].diff().dt.total_seconds()
# Set the time difference of the 1st row 0
df_val.loc[0, 'time_diff'] = 0
# Count number of processes per trace/ID
count_val_lst = df_val.groupby('case:concept:name').count()['time_diff'].tolist()
# Assign position number to each row/process
position_lst_1_val = [list(range(1, i + 1)) for i in count_val_lst]
position_lst_val = []
for i in position_lst_1_val:
    for j in i:
        position_lst_val.append(j)
df_val['position'] = position_lst_val
# Set the time difference of every process with position = 1 as 0
df_val.loc[df_val['position'] == 1, 'time_diff'] = 0
# Create the predicted time column per entry using the mean time difference
pred_time_lst_val = [mean_time_lst[j - 1] for j in position_lst_val]
df_val['baseline_predicted_time'] = pred_time_lst_val
df_val

,org:resource,lifecycle:transition,concept:name,time:timestamp,case:REG_DATE,case:concept:name,case:AMOUNT_REQ,time_diff,position,baseline_predicted_time
0,112.0,COMPLETE,A_SUBMITTED,2011-10-01 09:45:37.274000+02:00,2011-10-01 09:45:37.274000+02:00,173706,18000,0.000,1,0.557199
1,112.0,COMPLETE,A_PARTLYSUBMITTED,2011-10-01 09:45:37.363000+02:00,2011-10-01 09:45:37.274000+02:00,173706,18000,0.089,2,35.557763
2,112.0,SCHEDULE,W_Afhandelen leads,2011-10-01 09:46:18.623000+02:00,2011-10-01 09:45:37.274000+02:00,173706,18000,41.260,3,9691.845836
3,10912.0,START,W_Afhandelen leads,2011-10-01 10:15:43.883000+02:00,2011-10-01 09:45:37.274000+02:00,173706,18000,1765.260,4,11553.950361
4,10912.0,COMPLETE,A_PREACCEPTED,2011-10-01 10:16:48.956000+02:00,2011-10-01 09:45:37.274000+02:00,173706,18000,65.073,5,1227.997394
...,...,...,...,...,...,...,...,...,...,...
32245,10138.0,COMPLETE,W_Valideren aanvraag,2012-02-10 13:02:04.765000+01:00,2012-01-09 17:40:12.748000+01:00,199345,8500,1048.956,50,31357.574301
32246,10138.0,START,W_Valideren aanvraag,2012-02-10 13:32:48.693000+01:00,2012-01-09 17:40:12.748000+01:00,199345,8500,1843.928,51,34522.888947
32247,10138.0,COMPLETE,A_DECLINED,2012-02-10 13:41:38.050000+01:00,2012-01-09 17:40:12.748000+01:00,199345,8500,529.357,52,29883.379514
32248,10138.0,COMPLETE,O_DECLINED,2012-02-10 13:41:38.050000+01:00,2012-01-09 17:40:12.748000+01:00,199345,8500,0.000,53,35056.271916


In [13]:
# Calculate time difference
df_test['time_diff'] = df_test['time:timestamp'].diff().dt.total_seconds()
# Set the time difference of the 1st row 0
df_test.loc[0, 'time_diff'] = 0
# Count number of processes per trace/ID
count_test_lst = df_test.groupby('case:concept:name').count()['time_diff'].tolist()
# Assign position number to each row/process
position_lst_1_test = [list(range(1, i + 1)) for i in count_test_lst]
position_lst_test = []
for i in position_lst_1_test:
    for j in i:
        position_lst_test.append(j)
df_test['position'] = position_lst_test
# Set the time difference of every process with position = 1 as 0
df_test.loc[df_test['position'] == 1, 'time_diff'] = 0
# Create the predicted time column per entry using the mean time difference
pred_time_lst_test = [mean_time_lst[j - 1] for j in position_lst_test]
df_test['baseline_predicted_time'] = pred_time_lst_test
df_test

,org:resource,lifecycle:transition,concept:name,time:timestamp,case:REG_DATE,case:concept:name,case:AMOUNT_REQ,time_diff,position,baseline_predicted_time
0,112.0,COMPLETE,A_SUBMITTED,2012-01-09 17:44:18.199000+01:00,2012-01-09 17:44:18.199000+01:00,199348,6000,0.000,1,0.557199
1,112.0,COMPLETE,A_PARTLYSUBMITTED,2012-01-09 17:44:18.401000+01:00,2012-01-09 17:44:18.199000+01:00,199348,6000,0.202,2,35.557763
2,112.0,COMPLETE,A_PREACCEPTED,2012-01-09 17:44:55.414000+01:00,2012-01-09 17:44:18.199000+01:00,199348,6000,37.013,3,9691.845836
3,112.0,SCHEDULE,W_Completeren aanvraag,2012-01-09 17:44:55.822000+01:00,2012-01-09 17:44:18.199000+01:00,199348,6000,0.408,4,11553.950361
4,10929.0,START,W_Completeren aanvraag,2012-01-09 17:45:33+01:00,2012-01-09 17:44:18.199000+01:00,199348,6000,37.178,5,1227.997394
...,...,...,...,...,...,...,...,...,...,...
27257,112.0,COMPLETE,A_PARTLYSUBMITTED,2012-02-10 22:15:22.972000+01:00,2012-02-10 22:15:19.240000+01:00,208295,48000,3.732,2,35.557763
27258,112.0,COMPLETE,A_DECLINED,2012-02-10 22:16:18.231000+01:00,2012-02-10 22:15:19.240000+01:00,208295,48000,55.259,3,9691.845836
27259,112.0,COMPLETE,A_SUBMITTED,2012-02-10 22:57:49.541000+01:00,2012-02-10 22:57:49.541000+01:00,208301,2500,0.000,1,0.557199
27260,112.0,COMPLETE,A_PARTLYSUBMITTED,2012-02-10 22:57:49.866000+01:00,2012-02-10 22:57:49.541000+01:00,208301,2500,0.325,2,35.557763
